In [1]:
import os
import numpy as np
import torch
import scipy.io

In [12]:
COMPETITION_DATA_DIR = "/home/afan2025/Masters/Deep_Learning_Systems/competitionData"

In [130]:
os.listdir(os.path.join(COMPETITION_DATA_DIR, "train"))

['t12.2022.04.28.mat',
 't12.2022.06.02.mat',
 't12.2022.06.14.mat',
 't12.2022.08.25.mat',
 't12.2022.06.28.mat',
 't12.2022.05.19.mat',
 'train_names.txt',
 't12.2022.06.21.mat',
 't12.2022.05.17.mat',
 't12.2022.08.18.mat',
 't12.2022.08.11.mat',
 't12.2022.08.23.mat',
 't12.2022.05.24.mat',
 't12.2022.06.23.mat',
 't12.2022.06.16.mat',
 't12.2022.06.07.mat',
 't12.2022.07.29.mat',
 't12.2022.07.27.mat',
 't12.2022.07.05.mat',
 't12.2022.05.26.mat',
 't12.2022.08.13.mat',
 't12.2022.07.21.mat',
 't12.2022.05.05.mat',
 't12.2022.08.02.mat',
 't12.2022.07.14.mat']

In [98]:
with open(os.path.join(COMPETITION_DATA_DIR, "train", "train_names.txt"), 'rb') as f:
    train_names = f.read().decode('utf-8').splitlines()

with open(os.path.join(COMPETITION_DATA_DIR, "test", "test_names.txt"), 'rb') as f:
    test_names = f.read().decode('utf-8').splitlines()

with open(os.path.join(COMPETITION_DATA_DIR, "competitionHoldOut", "competitionHoldOut_names.txt"), 'rb') as f:
    holdout_train_names = f.read().decode('utf-8').splitlines()

In [132]:
#finding the maximum sequence length in all data
max_seq_length = -np.inf
sequence_lengths = []

for division in ["train","test", "competitionHoldOut"]:
    with open(os.path.join(COMPETITION_DATA_DIR, division, f"{division}_names.txt"), 'rb') as f:
        names = f.read().decode('utf-8').splitlines()
    for name in names:
        data_path = os.path.join(COMPETITION_DATA_DIR, division, name)
        dat = scipy.io.loadmat(data_path)
        # features = np.concatenate([dat['tx1'][0,i][:,0:128], dat['spikePow'][0,i][:,0:128]], axis=1)
        for i in range(dat['sentenceText'].shape[0]):    
            input_features = dat['tx1'][0,i] # Shape: (num_time_steps, 256)
            seq_length = input_features.shape[0]
            sequence_lengths.append(seq_length)
            max_seq_length = max(max_seq_length, seq_length)

print(np.mean(sequence_lengths))
max_seq_length

311.69834558823527


919

In [160]:
np.percentile(sequence_lengths, 50)

295.0

In [14]:
dat = scipy.io.loadmat(COMPETITION_DATA_DIR + '/train/t12.2022.04.28.mat')

In [15]:
dat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'sentenceText', 'tx1', 'tx2', 'tx3', 'tx4', 'spikePow', 'blockIdx'])

In [124]:
dat["tx1"][0,1].shape

(446, 256)

In [119]:
dat['blockIdx'].squeeze()

array([ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14,
       14, 14, 14, 14, 14

In [18]:
dat['sentenceText']

array(['Nuclear rockets can destroy airfields with ease.                                      ',
       'The best way to learn is to solve extra problems.                                     ',
       'The spray will be used in first division matches next season.                         ',
       "Our experiment's positive outcome was unexpected.                                     ",
       "Alimony harms a divorced man's wealth.                                                ",
       'She uses both names interchangeably.                                                  ',
       'The misquote was retracted with an apology.                                           ',
       'Critics fear the erosion of consumer protections and environmental standards.         ',
       'Her lips, moist and parted, spoke his name.                                           ',
       'How do they turn out later?                                                           ',
       'The eastern coast is a

In [20]:
input_features = []
transcriptions = []
frame_lens = []

for i in range(dat['sentenceText'].shape[0]):    
    #get time series of TX and spike power for this trial
    #first 128 columns = area 6v only
    features = np.concatenate([dat['tx1'][0,i][:,0:128], dat['spikePow'][0,i][:,0:128]], axis=1)

    sentence_len = features.shape[0]
    sentence = dat['sentenceText'][i].strip()

    input_features.append(features)
    transcriptions.append(sentence)
    frame_lens.append(sentence_len)

In [129]:
frame_lens

[478,
 446,
 554,
 461,
 348,
 382,
 481,
 641,
 391,
 365,
 591,
 484,
 591,
 751,
 494,
 473,
 617,
 718,
 593,
 765,
 715,
 315,
 725,
 401,
 736,
 394,
 517,
 403,
 543,
 579,
 595,
 859,
 786,
 811,
 437,
 369,
 474,
 807,
 527,
 508,
 802,
 845,
 904,
 901,
 845,
 766,
 453,
 781,
 759,
 364,
 630,
 827,
 697,
 603,
 515,
 454,
 800,
 549,
 590,
 829,
 513,
 859,
 683,
 457,
 832,
 472,
 684,
 738,
 848,
 893,
 589,
 738,
 481,
 480,
 445,
 514,
 583,
 694,
 427,
 821,
 493,
 835,
 400,
 537,
 536,
 451,
 458,
 274,
 859,
 427,
 498,
 899,
 906,
 518,
 496,
 549,
 562,
 488,
 749,
 516,
 872,
 904,
 874,
 803,
 227,
 474,
 337,
 511,
 627,
 808,
 300,
 511,
 767,
 770,
 574,
 638,
 850,
 608,
 673,
 858,
 546,
 516,
 446,
 518,
 687,
 363,
 618,
 488,
 664,
 586,
 505,
 494,
 506,
 486,
 439,
 401,
 529,
 578,
 390,
 829,
 416,
 850,
 843,
 436,
 580,
 611,
 615,
 450,
 801,
 441,
 588,
 467,
 578,
 505,
 497,
 401,
 522,
 759,
 756,
 444,
 378,
 768,
 451,
 871,
 465,
 609,
 782

In [45]:
input_features[5].shape

(382, 256)

In [28]:
len(transcriptions)

280

In [38]:
blockNums = np.squeeze(dat['blockIdx'])
blockList = np.unique(blockNums)
blocks = []
for b in range(len(blockList)):
    sentIdx = np.argwhere(blockNums==blockList[b])
    sentIdx = sentIdx[:,0].astype(np.int32)
    blocks.append(sentIdx)

for b in range(len(blocks)):
    feats = np.concatenate(input_features[blocks[b][0]:(blocks[b][-1]+1)], axis=0)
    print(feats.shape)
    feats_mean = np.mean(feats, axis=0, keepdims=True)
    feats_std = np.std(feats, axis=0, keepdims=True)
    for i in blocks[b]:
        input_features[i] = (input_features[i] - feats_mean) / (feats_std + 1e-8)

(10624, 256)
(11501, 256)
(13914, 256)
(12751, 256)
(11451, 256)
(12986, 256)
(10589, 256)
(11560, 256)
(11251, 256)
(10274, 256)
(9775, 256)
(10767, 256)
(10235, 256)
(10769, 256)


In [53]:
test = np.where(blockNums == blockList[0])[0]
test

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [37]:
blocks

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19], dtype=int32),
 array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39], dtype=int32),
 array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59], dtype=int32),
 array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
        77, 78, 79], dtype=int32),
 array([80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96,
        97, 98, 99], dtype=int32),
 array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119], dtype=int32),
 array([120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
        133, 134, 135, 136, 137, 138, 139], dtype=int32),
 array([140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159], dtype=int32),
 array([160, 161, 162, 163, 164, 165, 166, 167, 168,

In [69]:
input_features[0].shape

(478, 256)

In [71]:
input_features[0].reshape(-1, 128, 2).shape

(478, 128, 2)

In [67]:
columns = [
    [62, 60, 63, 58, 59, 61, 56, 57, 125, 123, 121, 119, 117, 115, 113, 127],
    [51, 53, 54, 55, 45, 49, 52, 50, 126, 124, 122, 120, 118, 116, 114, 111],
    [43, 41, 47, 48, 46, 42, 39, 37, 112, 110, 109, 108, 107, 106, 105, 104],
    [35, 33, 44, 40, 38, 36, 34, 32, 103, 102, 101, 100, 99, 97, 98, 96],
    [94, 95, 93, 92, 91, 90, 89, 88, 31, 29, 27, 25, 23, 21, 17, 30],
    [87, 86, 84, 85, 82, 83, 81, 80, 28, 26, 19, 15, 13, 20, 24, 22],
    [79, 77, 75, 73, 71, 69, 67, 65, 11, 9, 18, 12, 10, 7, 14, 16, ],
    [78, 76, 74, 72, 70, 68, 66, 64, 8, 5, 4, 6, 3, 2, 0, 1]
]

test = {}
for i in columns:
    print(len(i))
    for j in i:
        test[j] = test.get(j, 0) + 1
print(len(columns))

16
16
16
16
16
16
16
16
8


In [68]:
for i in range(128):
    if test[i] != 1:
        print(f"Channel {i} has count {test[i]}")

In [62]:
np.array(list(test.values())).shape

(128,)

In [74]:
rows = np.array(columns).T  # Shape: (8, 16)
rows

array([[ 62,  51,  43,  35,  94,  87,  79,  78],
       [ 60,  53,  41,  33,  95,  86,  77,  76],
       [ 63,  54,  47,  44,  93,  84,  75,  74],
       [ 58,  55,  48,  40,  92,  85,  73,  72],
       [ 59,  45,  46,  38,  91,  82,  71,  70],
       [ 61,  49,  42,  36,  90,  83,  69,  68],
       [ 56,  52,  39,  34,  89,  81,  67,  66],
       [ 57,  50,  37,  32,  88,  80,  65,  64],
       [125, 126, 112, 103,  31,  28,  11,   8],
       [123, 124, 110, 102,  29,  26,   9,   5],
       [121, 122, 109, 101,  27,  19,  18,   4],
       [119, 120, 108, 100,  25,  15,  12,   6],
       [117, 118, 107,  99,  23,  13,  10,   3],
       [115, 116, 106,  97,  21,  20,   7,   2],
       [113, 114, 105,  98,  17,  24,  14,   0],
       [127, 111, 104,  96,  30,  22,  16,   1]])

In [83]:
tester_reshaper = input_features[0][:, :128]
tester_reshaper[:, rows].shape

(478, 16, 8)

In [150]:
PHONE_DEF_SIL = [
    'AA', 'AE', 'AH', 'AO', 'AW',
    'AY', 'B',  'CH', 'D', 'DH',
    'EH', 'ER', 'EY', 'F', 'G',
    'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW',
    'OY', 'P', 'R', 'S', 'SH',
    'T', 'TH', 'UH', 'UW', 'V',
    'W', 'Y', 'Z', 'ZH', 'SIL', 'EOS'  # Add EOS
]

PHONE_TO_ID = {p: i for i, p in enumerate(PHONE_DEF_SIL)}

{'AA': 0,
 'AE': 1,
 'AH': 2,
 'AO': 3,
 'AW': 4,
 'AY': 5,
 'B': 6,
 'CH': 7,
 'D': 8,
 'DH': 9,
 'EH': 10,
 'ER': 11,
 'EY': 12,
 'F': 13,
 'G': 14,
 'HH': 15,
 'IH': 16,
 'IY': 17,
 'JH': 18,
 'K': 19,
 'L': 20,
 'M': 21,
 'N': 22,
 'NG': 23,
 'OW': 24,
 'OY': 25,
 'P': 26,
 'R': 27,
 'S': 28,
 'SH': 29,
 'T': 30,
 'TH': 31,
 'UH': 32,
 'UW': 33,
 'V': 34,
 'W': 35,
 'Y': 36,
 'Z': 37,
 'ZH': 38,
 'SIL': 39,
 'EOS': 40}

In [149]:
import re
for i, sentence in enumerate(transcriptions):
    sentence = re.sub(r'[^a-zA-Z\- \']', '', sentence) # Remove punctuation
    sentence = sentence.replace('--', '').lower()
    transcriptions[i] = sentence

transcriptions

['nuclear rockets can destroy airfields with ease',
 'the best way to learn is to solve extra problems',
 'the spray will be used in first division matches next season',
 "our experiment's positive outcome was unexpected",
 "alimony harms a divorced man's wealth",
 'she uses both names interchangeably',
 'the misquote was retracted with an apology',
 'critics fear the erosion of consumer protections and environmental standards',
 'her lips moist and parted spoke his name',
 'how do they turn out later',
 'the eastern coast is a place for pure pleasure and excitement',
 'please make your decision wisely to visit the beach',
 'she encouraged her children to make their own halloween costumes',
 'cleaned cloth must be protected against the redeposition of dispersed soil',
 'both figures would go higher in later years',
 'she always jokes about too much garlic in his food',
 'if people were more generous there would be no need for welfare',
 'the dimensions of these waves dwarf all our usua

In [154]:
from g2p_en import G2p
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')  # Also needed for tokenization
g2p = G2p()
for i, sentence in enumerate(transcriptions):
    phonemes = g2p(sentence)
    print(phonemes)
    break

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/afan2025/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /home/afan2025/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['N', 'UW1', 'K', 'L', 'IY0', 'ER0', ' ', 'R', 'AA1', 'K', 'AH0', 'T', 'S', ' ', 'K', 'AE1', 'N', ' ', 'D', 'IH0', 'S', 'T', 'R', 'OY1', ' ', 'EH1', 'R', 'F', 'IY2', 'L', 'D', 'Z', ' ', 'W', 'IH1', 'DH', ' ', 'IY1', 'Z']


In [ ]:
from tqdm import tqdm
phoneme_len_lists = []

for division in ["train","test", "competitionHoldOut"]:
    with open(os.path.join(COMPETITION_DATA_DIR, division, f"{division}_names.txt"), 'rb') as f:
        names = f.read().decode('utf-8').splitlines()
    for name in names:
        print("Processing file in",division, name)
        data_path = os.path.join(COMPETITION_DATA_DIR, division, name)
        dat = scipy.io.loadmat(data_path)
        # features = np.concatenate([dat['tx1'][0,i][:,0:128], dat['spikePow'][0,i][:,0:128]], axis=1)
        for i in tqdm(range(dat['sentenceText'].shape[0])):    
            sentence = dat['sentenceText'][i].strip()
            sentence = re.sub(r'[^a-zA-Z\- \']', '', sentence) # Remove punctuation
            sentence = sentence.replace('--', '').lower()
            phonemes = g2p(sentence)
            phoneme_len_lists.append(len(phonemes))

print(np.mean(phoneme_len_lists))
print(np.percentile(phoneme_len_lists, 99))

In [162]:
print(np.mean(phoneme_len_lists))
print(np.percentile(phoneme_len_lists, 99))

24.843198529411765
62.0


In [156]:
max(phoneme_len_lists)

77

In [ ]:
from tqdm import tqdm
char_len_lists = []
word_len_lists = []

for division in ["train","test", "competitionHoldOut"]:
    with open(os.path.join(COMPETITION_DATA_DIR, division, f"{division}_names.txt"), 'rb') as f:
        names = f.read().decode('utf-8').splitlines()
    for name in names:
        print("Processing file in",division, name)
        data_path = os.path.join(COMPETITION_DATA_DIR, division, name)
        dat = scipy.io.loadmat(data_path)
        # features = np.concatenate([dat['tx1'][0,i][:,0:128], dat['spikePow'][0,i][:,0:128]], axis=1)
        for i in tqdm(range(dat['sentenceText'].shape[0])):    
            sentence = dat['sentenceText'][i].strip()
            sentence = re.sub(r'[^a-zA-Z\- \']', '', sentence) # Remove punctuation
            sentence = sentence.replace('--', '').lower()
            char_len_lists.append(len(sentence))
            word_len_lists.append(len(sentence.split()))

print(np.mean(char_len_lists))
print(np.percentile(char_len_lists, 99))
print(np.mean(word_len_lists))
print(np.percentile(word_len_lists, 99))

In [189]:
print(np.mean(char_len_lists))
print(np.percentile(char_len_lists, 10))
print(np.mean(word_len_lists))
print(np.percentile(word_len_lists, 99))
print(np.max(char_len_lists))
print(np.max(word_len_lists))

29.267003676470587
8.0
5.838143382352941
12.0
87
18


# LibriSpeech ASR Dataset EDA

In [168]:
pwd

'/home/afan2025'

In [197]:
import urllib.request
import tarfile
import gzip
from io import BytesIO

url = "https://www.openslr.org/resources/12/train-clean-360.tar.gz"
tar_file = "/home/afan2025/Masters/Deep_Learning_Systems/libriSpeechASR/train-clean-360.tar.gz"
output_file = "/home/afan2025/Masters/Deep_Learning_Systems/libriSpeechASR/libriSpeechASR_text.txt"

# Step 1: Download (with progress)
print("Downloading tar file...")
def download_progress(block_num, block_size, total_size):
    downloaded = block_num * block_size
    percent = (downloaded / total_size) * 100 if total_size > 0 else 0
    print(f"\rProgress: {percent:.1f}% ({downloaded / 1e9:.2f} GB / {total_size / 1e9:.2f} GB)", end='')

urllib.request.urlretrieve(url, tar_file, reporthook=download_progress)
print("\nDownload complete!")

Progress: 100.0% (23.05 GB / 23.05 GB)
Download complete!


In [206]:
MIN_CHARS = 0
MAX_CHARS = 200

# Step 2: Extract text (much faster from local file)
print("Extracting text from tar file...")
line_count = 0
filtered_count = 0

with open(output_file, 'w', encoding='utf-8') as outfile:
    with tarfile.open(tar_file, 'r:gz') as tar:
        for member in tar.getmembers():
            if member.name.endswith('.trans.txt'):
                f = tar.extractfile(member)
                for line in f:
                    parts = line.decode('utf-8').strip().split(' ', 1)
                    if len(parts) == 2:
                        text = parts[1].lower()
                        text = re.sub(r'[^a-zA-Z\- \']', '', text)
                        if MIN_CHARS <= len(text) and len(text) <= MAX_CHARS:
                            print("conditional achieved")
                            outfile.write(text + '\n')
                            filtered_count += 1
                            if filtered_count % 1000 == 0:
                                outfile.flush()  # Force write to disk
                                print(f"  {filtered_count} lines saved...")
                        line_count += 1


print(f"\nExtraction complete!")
print(f"Saved {filtered_count} lines (out of {line_count} total)")

Extracting text from tar file...
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
conditional achieved
c

In [2]:
from g2p_en import G2p
with open("/home/afan2025/Masters/Deep_Learning_Systems/libriSpeechASR/libriSpeechASR_text.txt", "r") as f:
    lines = f.readlines()

phoneme_lens = []

g2p = G2p()
# Grab 10k random indices
import random
random_indices = random.sample(range(len(lines)), min(5000, len(lines)))
random_lines = [lines[i].strip() for i in random_indices]

for line in random_lines:
    phoneme_lens.append(len(g2p(line)))


In [17]:
np.percentile(phoneme_lens, 45)

130.0

In [3]:
max(phoneme_lens)

186